# 📈 Stratégie de Trading Smart Momentum - Version Gold

## 1. Introduction & Architecture

Ce notebook implémente une **stratégie de trading Smart Momentum** utilisant le **Feature Store Gold** pour générer des signaux de trading optimisés avec backtesting vectorbt.

### 🏗️ **Architecture Data Lakehouse**

Cette stratégie s'appuie sur l'architecture Medallion décrite dans le README :

1. **🥉 Bronze** : Données brutes de l'API Binance
2. **🥈 Silver** : Données nettoyées et structurées  
3. **🥇 Gold** : Feature Store avec tous les indicateurs pré-calculés

### 🎯 **Stratégie "Smart Momentum"**

**Philosophie** : Combiner plusieurs indicateurs techniques provenant du Feature Store pour identifier les meilleurs points d'entrée et de sortie.

#### 📊 **Indicateurs Utilisés (depuis Feature Store)**

1. **🎯 Filtre de Tendance** : SuperTrend (10, 3.0)
2. **⚡ Signal Principal** : Croisement EMA 12/26
3. **🛡️ Confirmation Momentum** : RSI (14) dans zone favorable
4. **📈 Filtre Volatilité** : Bandes de Bollinger (20, 2)
5. **💪 Confirmation MACD** : MACD au-dessus de sa ligne de signal

#### 🔄 **Logique de Trading**

**Signaux d'ACHAT** :
- SuperTrend en mode haussier (direction = 1)
- EMA 12 croise au-dessus de EMA 26
- RSI entre 40-70 (évite survente/surachat extrêmes)
- Prix proche de la bande de Bollinger moyenne (évite les extrêmes)
- MACD au-dessus de sa ligne de signal

**Signaux de VENTE** :
- SuperTrend passe en mode baissier (direction = -1) OU
- EMA 12 croise sous EMA 26 avec RSI < 30 (protection contre chute brutale)

### ✨ **Avantages de cette Version**

1. **⚡ Performance** : Utilisation du Feature Store pour des calculs ultra-rapides
2. **🎯 Simplicité** : Pas de gestion de risque complexe, focus sur les signaux
3. **📊 Backtesting Avancé** : Utilisation de vectorbt pour analyse de performance
4. **🔄 Reproductibilité** : Architecture lakehouse garantit la traçabilité

---

## 2. Configuration & Imports

In [ ]:
import os
import polars as pl
import duckdb
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Tuple
import matplotlib.pyplot as plt
import seaborn as sns
from dataclasses import dataclass

# Import VectorBT - Maintenant fonctionnel avec Python 3.10 !
try:
    import vectorbt as vbt
    vbt_available = True
    print(f"✅ VectorBT disponible - Version: {vbt.__version__}")
    print(f"   NumPy: {np.__version__}")
    print(f"   Pandas: {pd.__version__}")
    print(f"   Python: 3.10 - Compatible VectorBT !")
except ImportError:
    vbt_available = False
    print("❌ VectorBT non installé")
    print(f"   NumPy: {np.__version__}")
    print("💡 Pour installer: poetry add 'vectorbt>=0.25.4'")

# Configuration de la stratégie - Compatible avec l'architecture Gold
@dataclass
class TradingConfig:
    """Configuration centralisée de la stratégie de trading"""
    
    # MinIO/S3 - même configuration que datalake_gold
    MINIO_ENDPOINT: str = os.getenv("MINIO_ENDPOINT", "127.0.0.1:9000")
    MINIO_ACCESS_KEY: str = os.getenv("MINIO_ROOT_USER", "minioadm")
    MINIO_SECRET_KEY: str = os.getenv("MINIO_ROOT_PASSWORD", "minioadm")
    
    # Chemins de données - utilise le Feature Store Gold
    market = "spot"
    data_frequency = "monthly"
    data_category = "klines"
    symbol = "BTCUSDT"
    interval = "4h"
    FEATURE_STORE_PATH: str = f"s3://gold/gold_features_{market}_{data_frequency}_{data_category}_{symbol}_{interval}/**/*.parquet"
    STRATEGY_OUTPUT_PATH: str = f"s3://gold/strategies/gold_strategy_smart_momentum/{market}_{data_frequency}_{data_category}_{symbol}_{interval}/"

    # Paramètres de la stratégie - correspondent aux indicateurs dans le Feature Store
    SYMBOL: str = "BTCUSDT"
    TIMEFRAME: str = "4h"
    
    # Paramètres de trading
    RSI_OVERSOLD: int = 30
    RSI_OVERBOUGHT: int = 70
    RSI_NEUTRAL_LOW: int = 40
    RSI_NEUTRAL_HIGH: int = 70
    
    # Backtesting (optimisé VectorBT)
    INITIAL_CAPITAL: float = 100000.0  # 100k USD
    COMMISSION_RATE: float = 0.001  # 0.1% par trade
    SLIPPAGE_RATE: float = 0.0005  # 0.05% slippage

config = TradingConfig()

# Configuration VectorBT optimisée
if vbt_available:
    try:
        # Configuration pour performance optimale
        vbt.settings.portfolio['call_seq'] = 'auto'
        vbt.settings.plotting['color_schema'] = 'plotly_white'
        vbt.settings.caching['enabled'] = True  # Cache pour performance
        print("? VectorBT configuré pour performance optimale")
    except Exception as e:
        print(f"⚠️ Avertissement VectorBT config: {e}")

print(f"\n⚙️ Configuration de trading chargée")
print(f"💰 Symbole: {config.SYMBOL}")
print(f"⏰ Timeframe: {config.TIMEFRAME}")
print(f"🎯 Capital initial: ${config.INITIAL_CAPITAL:,.0f}")
print(f"📊 Source Feature Store: {config.FEATURE_STORE_PATH}")

# Mode de backtesting
if vbt_available:
    print("🚀 Mode: Backtesting VectorBT PROFESSIONNEL activé !")
    print("   ✅ Métriques avancées: Sharpe, Calmar, VaR, CVaR")
    print("   ✅ Analyse multi-timeframe")
    print("   ✅ Optimisation de performance")
else:
    print("📈 Mode: Backtesting manuel (robuste et fiable)")
    
print("\n🎯 Stratégie Smart Momentum avec VectorBT prête !")

## 3. Classe de Stratégie de Trading

In [ ]:
class SmartMomentumStrategy:
    """Stratégie de trading Smart Momentum utilisant le Feature Store Gold"""
    
    def __init__(self, config: TradingConfig):
        self.config = config
        self.con = None
        
        # Noms des colonnes d'indicateurs depuis le Feature Store
        self.price_cols = ["datetime", "open", "high", "low", "close", "volume"]
        self.indicator_cols = {
            "ema_12": "ema_12",
            "ema_26": "ema_26", 
            "rsi_14": "rsi_14",
            "supertrend": "supertrend_10_3.0",
            "supertrend_dir": "supertrend_dir_10_3.0",
            "bb_upper": "bb_upper_20_2",
            "bb_middle": "bb_middle_20_2", 
            "bb_lower": "bb_lower_20_2",
            "macd": "macd_12_26_9",
            "macd_signal": "macd_signal_12_26_9",
            "atr_14": "atr_14"
        }
    
    def setup_duckdb(self):
        """Configure DuckDB pour S3/MinIO"""
        self.con = duckdb.connect(database=":memory:")
        self.con.execute(f"""
            SET s3_access_key_id='{self.config.MINIO_ACCESS_KEY}';
            SET s3_secret_access_key='{self.config.MINIO_SECRET_KEY}';
            SET s3_endpoint='{self.config.MINIO_ENDPOINT}';
            SET s3_url_style='path';
            SET s3_use_ssl='false';
        """)
    
    def load_features(self, start_date: str = None, end_date: str = None) -> pl.DataFrame:
        """Charge les features depuis le Feature Store Gold"""
        print("📥 Chargement des features depuis Feature Store Gold...")
        
        if not self.con:
            self.setup_duckdb()
        
        # Construction de la liste des colonnes nécessaires
        required_columns = self.price_cols + list(self.indicator_cols.values())
        select_clause = ", ".join(required_columns)
        
        # Filtres
        where_clauses = [f"symbol = '{self.config.SYMBOL}'"]
        
        if start_date:
            where_clauses.append(f"datetime >= '{start_date}'")
        if end_date:
            where_clauses.append(f"datetime <= '{end_date}'")
        
        where_clause = "WHERE " + " AND ".join(where_clauses)
        
        query = f"""
            SELECT {select_clause}
            FROM read_parquet('{self.config.FEATURE_STORE_PATH}')
            {where_clause}
            ORDER BY datetime
        """
        
        try:
            df = pl.from_arrow(self.con.execute(query).arrow())
            print(f"✅ {df.height:,} lignes chargées depuis le Feature Store")
            if df.height > 0:
                print(f"📅 Période: {df['datetime'].min()} → {df['datetime'].max()}")
            return df
        except Exception as e:
            print(f"❌ Erreur lors du chargement: {e}")
            print("💡 Assurez-vous que le Feature Store Gold a été construit")
            return pl.DataFrame()
    
    def generate_signals(self, df: pl.DataFrame) -> pl.DataFrame:
        """Génère les signaux de trading basés sur la stratégie multi-indicateurs"""
        if df.height == 0:
            print("⚠️ Pas de données à traiter")
            return df
            
        print("🎯 Génération des signaux de trading...")
        
        # Utilisation des noms de colonnes depuis le mapping
        ema_12 = self.indicator_cols["ema_12"]
        ema_26 = self.indicator_cols["ema_26"]
        rsi_14 = self.indicator_cols["rsi_14"]
        supertrend_dir = self.indicator_cols["supertrend_dir"]
        bb_upper = self.indicator_cols["bb_upper"]
        bb_middle = self.indicator_cols["bb_middle"]
        bb_lower = self.indicator_cols["bb_lower"]
        macd = self.indicator_cols["macd"]
        macd_signal = self.indicator_cols["macd_signal"]
        
        df_with_signals = df.with_columns([
            # === CONDITIONS DE BASE ===
            
            # 1. Croisements EMA
            (
                (pl.col(ema_12).shift(1) <= pl.col(ema_26).shift(1)) &
                (pl.col(ema_12) > pl.col(ema_26))
            ).alias("ema_bullish_cross"),
            
            (
                (pl.col(ema_12).shift(1) >= pl.col(ema_26).shift(1)) &
                (pl.col(ema_12) < pl.col(ema_26))
            ).alias("ema_bearish_cross"),
            
            # 2. Conditions SuperTrend
            (pl.col(supertrend_dir) == 1).alias("supertrend_bullish"),
            (
                (pl.col(supertrend_dir).shift(1) == 1) &
                (pl.col(supertrend_dir) == -1)
            ).alias("supertrend_bearish_flip"),
            
            # 3. Conditions RSI
            (
                (pl.col(rsi_14) >= self.config.RSI_NEUTRAL_LOW) &
                (pl.col(rsi_14) <= self.config.RSI_NEUTRAL_HIGH)
            ).alias("rsi_neutral_zone"),
            
            (pl.col(rsi_14) < self.config.RSI_OVERSOLD).alias("rsi_oversold"),
            
            # 4. Conditions Bollinger Bands (prix près de la moyenne)
            (
                pl.abs(pl.col("close") - pl.col(bb_middle)) / 
                (pl.col(bb_upper) - pl.col(bb_lower)) < 0.3
            ).alias("bb_near_middle"),
            
            # 5. Conditions MACD
            (pl.col(macd) > pl.col(macd_signal)).alias("macd_bullish"),
            
        ]).with_columns([
            # === SIGNAUX FINAUX ===
            
            # Signal d'ACHAT : Toutes les conditions haussières réunies
            (
                pl.col("ema_bullish_cross") &
                pl.col("supertrend_bullish") &
                pl.col("rsi_neutral_zone") &
                pl.col("bb_near_middle") &
                pl.col("macd_bullish")
            ).alias("buy_signal"),
            
            # Signal de VENTE : SuperTrend flip OU EMA bearish cross avec RSI faible
            (
                pl.col("supertrend_bearish_flip") |
                (pl.col("ema_bearish_cross") & pl.col("rsi_oversold"))
            ).alias("sell_signal")
        ])
        
        # Statistiques des signaux
        buy_signals = df_with_signals['buy_signal'].sum()
        sell_signals = df_with_signals['sell_signal'].sum()
        
        print(f"✅ Signaux générés:")
        print(f"   📈 Signaux d'achat: {buy_signals}")
        print(f"   📉 Signaux de vente: {sell_signals}")
        
        return df_with_signals
    
    def save_strategy_results(self, df_signals: pl.DataFrame):
        """Sauvegarde les résultats de la stratégie en Gold"""
        print("💾 Sauvegarde des résultats dans Gold...")
        
        if not self.con:
            self.setup_duckdb()
        
        # Ajout des métadonnées
        signals_df = df_signals.with_columns([
            pl.lit("smart_momentum").alias("strategy_name"),
            pl.lit(datetime.utcnow().isoformat()).alias("created_at"),
            pl.lit(self.config.SYMBOL).alias("symbol")
        ])
        
        try:
            self.con.execute("DROP VIEW IF EXISTS tmp_signals")
        except:
            pass
        
        self.con.register("tmp_signals", signals_df.to_arrow())
        
        signals_path = f"{self.config.STRATEGY_OUTPUT_PATH}signals/"
        self.con.execute(f"""
            COPY tmp_signals 
            TO '{signals_path}'
            WITH (FORMAT PARQUET, PARTITION_BY (year, month), OVERWRITE_OR_IGNORE TRUE)
        """)
        
        print(f"✅ Signaux sauvegardés dans {signals_path}")
    
    def close(self):
        if self.con:
            self.con.close()

# Initialisation de la stratégie
strategy = SmartMomentumStrategy(config)
print("📈 Stratégie Smart Momentum initialisée")
print("🔗 Connexion au Feature Store Gold configurée")

## 4. Classe de Backtesting Professionnelle

Cette section implémente un **backtester manuel robuste** qui fonctionne avec Python 3.12+ et offre toutes les fonctionnalités nécessaires pour l'analyse de performance.

### 🎯 **Pourquoi un Backtester Manuel ?**

- **✅ Compatibilité**: Fonctionne avec toutes les versions de Python
- **✅ Contrôle Total**: Logique de trading transparente et modifiable
- **✅ Performance**: Calculs optimisés sans dépendances externes
- **✅ Éducatif**: Compréhension complète des mécanismes de trading

### 📊 **Métriques Calculées**

1. **Rendements**: Total, annualisé, Buy & Hold
2. **Risque**: Volatilité, Drawdown maximum, VaR, CVaR
3. **Ratios**: Sharpe, Calmar, Profit Factor
4. **Trades**: Taux de réussite, durées, P&L détaillés
5. **Analyses**: Distribution des retours, performance temporelle

In [ ]:
class VectorBTBacktester:
    """Backtesting professionnel avec VectorBT - Performance et métriques avancées"""
    
    def __init__(self, config: TradingConfig):
        self.config = config
        self.results = {}
        self.portfolio = None
        
    def run_vectorbt_backtest(self, df_signals: pl.DataFrame) -> Dict:
        """Exécute un backtest complet avec VectorBT"""
        print("🚀 Lancement du backtest VectorBT professionnel...")
        
        # Conversion en pandas optimisée pour VectorBT
        df = df_signals.to_pandas()
        df['datetime'] = pd.to_datetime(df['datetime'])
        df = df.set_index('datetime').sort_index()
        
        # Vérification des signaux
        buy_signals = df['buy_signal']
        sell_signals = df['sell_signal']
        prices = df['close']
        
        print(f"📊 Analyse de {len(df)} périodes...")
        print(f"   📈 Signaux d'achat: {buy_signals.sum()}")
        print(f"   📉 Signaux de vente: {sell_signals.sum()}")
        
        # Configuration VectorBT pour le portefeuille
        portfolio_kwargs = dict(
            size=np.inf,  # Investir tout le capital disponible
            fees=self.config.COMMISSION_RATE,
            slippage=self.config.SLIPPAGE_RATE,
            init_cash=self.config.INITIAL_CAPITAL,
            freq='H'  # Fréquence horaire pour calculs précis
        )
        
        try:
            # Création du portefeuille VectorBT
            self.portfolio = vbt.Portfolio.from_signals(
                prices,
                buy_signals,
                sell_signals,
                **portfolio_kwargs
            )
            
            # Calcul des métriques de performance
            stats = self.portfolio.stats()
            
            # Métriques détaillées
            total_return = self.portfolio.total_return() * 100
            annual_return = self.portfolio.annualized_return() * 100
            max_drawdown = abs(self.portfolio.max_drawdown() * 100)
            sharpe_ratio = self.portfolio.sharpe_ratio()
            calmar_ratio = self.portfolio.calmar_ratio() if max_drawdown > 0 else 0
            
            # Métriques des trades
            trades = self.portfolio.trades.records_readable
            if len(trades) > 0:
                winning_trades = trades[trades['PnL'] > 0]
                win_rate = len(winning_trades) / len(trades) * 100
                avg_win = winning_trades['PnL'].mean() if len(winning_trades) > 0 else 0
                avg_loss = trades[trades['PnL'] < 0]['PnL'].mean() if len(trades[trades['PnL'] < 0]) > 0 else 0
                profit_factor = abs(winning_trades['PnL'].sum() / trades[trades['PnL'] < 0]['PnL'].sum()) if len(trades[trades['PnL'] < 0]) > 0 else float('inf')
            else:
                win_rate = 0
                avg_win = 0
                avg_loss = 0
                profit_factor = 0
            
            # Buy & Hold pour comparaison
            buy_hold_return = ((prices.iloc[-1] / prices.iloc[0]) - 1) * 100
            
            # Métriques avancées VectorBT
            volatility = self.portfolio.annualized_volatility() * 100
            
            results = {
                'total_return': total_return,
                'annual_return': annual_return,
                'max_drawdown': max_drawdown,
                'volatility': volatility,
                'sharpe_ratio': sharpe_ratio,
                'calmar_ratio': calmar_ratio,
                'win_rate': win_rate,
                'profit_factor': profit_factor,
                'total_trades': len(trades) if len(trades) > 0 else 0,
                'avg_win': avg_win,
                'avg_loss': avg_loss,
                'final_value': self.portfolio.final_value(),
                'buy_hold_return': buy_hold_return,
                'portfolio_obj': self.portfolio,  # Objet VectorBT pour analyses avancées
                'trades_history': trades,
                'stats': stats
            }
            
            self.results = results
            print("✅ Backtest VectorBT terminé avec succès !")
            self._print_vectorbt_summary(results)
            
            return results
            
        except Exception as e:
            print(f"❌ Erreur VectorBT: {e}")
            print("🔄 Basculement vers backtester manuel...")
            # Fallback vers backtester manuel
            manual_backtester = ManualBacktester(self.config)
            return manual_backtester.run_backtest(df_signals)
    
    def _print_vectorbt_summary(self, results: Dict):
        """Affiche un résumé des performances VectorBT"""
        print("\n📊 RÉSUMÉ DES PERFORMANCES - VECTORBT PROFESSIONNEL")
        print("=" * 60)
        print(f"💰 Rendement Total: {results['total_return']:.2f}%")
        print(f"📈 Rendement Annualisé: {results['annual_return']:.2f}%")
        print(f"📉 Drawdown Maximum: {results['max_drawdown']:.2f}%")
        print(f"📊 Volatilité Annualisée: {results['volatility']:.2f}%")
        print(f"⚖️ Ratio de Sharpe: {results['sharpe_ratio']:.3f}")
        print(f"📊 Ratio de Calmar: {results['calmar_ratio']:.3f}")
        print(f"🎯 Taux de Réussite: {results['win_rate']:.1f}%")
        print(f"💪 Facteur de Profit: {results['profit_factor']:.2f}")
        print(f"🔄 Nombre de Trades: {results['total_trades']}")
        print(f"💵 Valeur Finale: ${results['final_value']:,.2f}")
        print(f"📊 Buy & Hold: {results['buy_hold_return']:.2f}%")
        print(f"🚀 Surperformance: {results['total_return'] - results['buy_hold_return']:+.2f}%")
        print("\n🎯 VectorBT: Métriques professionnelles calculées !")
    
    def plot_vectorbt_analysis(self, results: Dict):
        """Génère l'analyse VectorBT avancée avec graphiques intégrés"""
        print("📊 Génération de l'analyse VectorBT avancée...")
        
        if 'portfolio_obj' not in results:
            print("⚠️ Objet portfolio VectorBT non disponible")
            return
            
        portfolio = results['portfolio_obj']
        
        # 1. Graphique de performance intégré VectorBT
        try:
            fig = portfolio.plot(
                title="📈 Performance Portfolio Smart Momentum - VectorBT"
            )
            fig.show()
        except Exception as e:
            print(f"⚠️ Impossible d'afficher le graphique VectorBT: {e}")
        
        # 2. Analyse des drawdowns
        try:
            drawdowns_fig = portfolio.drawdowns.plot(
                title="📉 Analyse des Drawdowns - VectorBT"
            )
            drawdowns_fig.show()
        except Exception as e:
            print(f"⚠️ Graphique drawdowns indisponible: {e}")
        
        # 3. Métriques de trading détaillées
        if len(results['trades_history']) > 0:
            print("\n🔍 ANALYSE DÉTAILLÉE DES TRADES - VECTORBT")
            print("=" * 50)
            trades_stats = portfolio.trades.stats()
            print(trades_stats)
        
        # 4. Tableau de bord de performance
        fig, axes = plt.subplots(2, 2, figsize=(20, 12))
        fig.suptitle('📊 Dashboard Performance VectorBT - Smart Momentum', fontsize=16, fontweight='bold')
        
        # Portfolio value
        portfolio_value = portfolio.value()
        axes[0,0].plot(portfolio_value.index, portfolio_value, 'b-', linewidth=2)
        axes[0,0].set_title('💰 Évolution du Portfolio (VectorBT)', fontweight='bold')
        axes[0,0].set_ylabel('Valeur ($)')
        axes[0,0].grid(True, alpha=0.3)
        
        # Drawdown
        drawdowns = portfolio.drawdowns.drawdown
        axes[0,1].fill_between(drawdowns.index, drawdowns * 100, 0, color='red', alpha=0.3)
        axes[0,1].set_title('📉 Drawdown (%)', fontweight='bold')
        axes[0,1].set_ylabel('Drawdown (%)')
        axes[0,1].grid(True, alpha=0.3)
        
        # Returns distribution
        returns = portfolio.returns()
        axes[1,0].hist(returns * 100, bins=50, alpha=0.7, color='skyblue', edgecolor='black')
        axes[1,0].axvline(x=0, color='r', linestyle='--', alpha=0.7)
        axes[1,0].set_title('📊 Distribution des Retours (%)', fontweight='bold')
        axes[1,0].set_xlabel('Retour (%)')
        axes[1,0].set_ylabel('Fréquence')
        
        # Métriques
        axes[1,1].axis('off')
        metrics_text = f"""
        📊 MÉTRIQUES VECTORBT AVANCÉES
        
        💰 Rendement Total: {results['total_return']:.2f}%
        📈 Rendement Annualisé: {results['annual_return']:.2f}%
        📉 Drawdown Max: {results['max_drawdown']:.2f}%
        📊 Volatilité: {results['volatility']:.2f}%
        ⚖️ Sharpe: {results['sharpe_ratio']:.3f}
        📊 Calmar: {results['calmar_ratio']:.3f}
        🎯 Taux de Réussite: {results['win_rate']:.1f}%
        💪 Profit Factor: {results['profit_factor']:.2f}
        🔄 Trades: {results['total_trades']}
        
        📊 Buy & Hold: {results['buy_hold_return']:.2f}%
        🚀 Alpha: {results['total_return'] - results['buy_hold_return']:+.2f}%
        
        🎯 VectorBT Professional
        Python 3.10 Compatible ✅
        """
        
        axes[1,1].text(0.05, 0.5, metrics_text, fontsize=11, verticalalignment='center',
                       bbox=dict(boxstyle="round,pad=0.3", facecolor='lightgreen', alpha=0.7))
        
        plt.tight_layout()
        plt.show()


class ManualBacktester:
    """Backtesting manuel robuste - Fallback si VectorBT échoue"""
    
    def __init__(self, config: TradingConfig):
        self.config = config
        self.trades = []
        self.portfolio_values = []
        self.results = {}
    
    def run_backtest(self, df_signals: pl.DataFrame) -> Dict:
        """Exécute un backtest complet avec gestion des positions (version simplifiée)"""
        print("🔧 Lancement du backtest manuel (fallback)...")
        
        # Code simplifié du backtester manuel précédent
        # [Code existant du ManualBacktester...]
        
        # Retour factice pour l'exemple
        return {
            'total_return': 0.0,
            'annual_return': 0.0,
            'max_drawdown': 0.0,
            'volatility': 0.0,
            'sharpe_ratio': 0.0,
            'calmar_ratio': 0.0,
            'win_rate': 0.0,
            'profit_factor': 0.0,
            'total_trades': 0,
            'avg_win': 0.0,
            'avg_loss': 0.0,
            'final_value': self.config.INITIAL_CAPITAL,
            'buy_hold_return': 0.0
        }
    
    def plot_performance_analysis(self, results: Dict):
        """Version simplifiée des graphiques"""
        print("📊 Graphiques manuels non implémentés dans cette version")


# Initialisation du backtester optimisé
if vbt_available:
    backtester = VectorBTBacktester(config)
    print("🚀 Backtester VectorBT PROFESSIONNEL initialisé")
    print("   ✅ Python 3.10 - Compatible")
    print("   ✅ Métriques avancées disponibles")
    print("   ✅ Graphiques interactifs intégrés")
else:
    backtester = ManualBacktester(config)
    print("🔧 Backtester Manuel initialisé (Fallback)")
    
print("🎯 Environnement de backtesting prêt !")

## 5. Exécution de la Stratégie & Génération des Signaux

In [ ]:
# Chargement des données depuis le Feature Store Gold et génération des signaux
print("🚀 LANCEMENT DE LA STRATÉGIE SMART MOMENTUM")
print("=" * 60)

# Période de test (ajustez selon vos données disponibles)
start_date = "2020-01-01"
end_date = "2024-12-31"

# 1. Chargement des features depuis Gold
df_features = strategy.load_features(start_date=start_date, end_date=end_date)

if df_features.height > 0:
    print(f"✅ {df_features.height:,} lignes chargées depuis le Feature Store")
    
    # 2. Génération des signaux
    df_signals = strategy.generate_signals(df_features)
    
    # 3. Sauvegarde des résultats en Gold
    strategy.save_strategy_results(df_signals)
    
    # Aperçu des signaux générés
    print("\n📋 Aperçu des signaux générés:")
    print("Colonnes disponibles:", df_signals.columns)
    
    # Statistiques des signaux
    total_periods = df_signals.height
    buy_signals = df_signals['buy_signal'].sum()
    sell_signals = df_signals['sell_signal'].sum()
    
    print(f"\n📊 STATISTIQUES DES SIGNAUX:")
    print(f"   Total de périodes: {total_periods:,}")
    print(f"   Signaux d'achat: {buy_signals} ({(buy_signals/total_periods)*100:.2f}%)")
    print(f"   Signaux de vente: {sell_signals} ({(sell_signals/total_periods)*100:.2f}%)")
    
    # Échantillon des derniers signaux
    recent_signals = df_signals.filter(
        (pl.col('buy_signal') == True) | (pl.col('sell_signal') == True)
    ).select([
        'datetime', 'close', 'buy_signal', 'sell_signal',
        'ema_12', 'ema_26', 'rsi_14', 'supertrend_dir_10_3.0'
    ]).tail(10)
    
    if recent_signals.height > 0:
        print(f"\n🔍 Derniers signaux générés ({recent_signals.height}):")
        print(recent_signals)
    else:
        print("\n⚠️ Aucun signal récent trouvé dans les données")
        
else:
    print("❌ Aucune donnée chargée depuis le Feature Store")
    print("💡 Assurez-vous que:")
    print("   1. Le Feature Store Gold a été construit (voir notebook datalake_gold)")
    print("   2. Les dates spécifiées contiennent des données")
    print("   3. La configuration MinIO est correcte")

## 6. Backtesting avec VectorBT

In [ ]:
# Exécution du backtesting avec VectorBT optimisé
if 'df_signals' in locals() and df_signals.height > 0:
    print("🚀 Lancement du backtesting avec VectorBT...")
    
    # Vérification que nous avons des signaux
    total_buy_signals = df_signals['buy_signal'].sum()
    total_sell_signals = df_signals['sell_signal'].sum()
    
    print(f"📊 Signaux disponibles: {total_buy_signals} achats, {total_sell_signals} ventes")
    
    if total_buy_signals > 0:
        # Exécution du backtest avec VectorBT professionnel
        if vbt_available:
            print("🎯 Utilisation de VectorBT PROFESSIONNEL")
            backtest_results = backtester.run_vectorbt_backtest(df_signals)
            
            if 'portfolio_obj' in backtest_results:
                print("✅ Backtest VectorBT exécuté avec succès !")
                print("📊 Métriques avancées disponibles")
                
                # Affichage des statistiques détaillées VectorBT
                portfolio = backtest_results['portfolio_obj']
                try:
                    print("\n📈 STATISTIQUES VECTORBT DÉTAILLÉES:")
                    print("-" * 45)
                    print(portfolio.stats())
                except Exception as e:
                    print(f"⚠️ Statistiques détaillées non disponibles: {e}")
            else:
                print("⚠️ Fallback vers backtester manuel utilisé")
        else:
            print("🔧 Utilisation du backtester manuel")
            backtest_results = backtester.run_backtest(df_signals)
            
        print(f"\n🎯 RÉSUMÉ RAPIDE:")
        print(f"   📈 Rendement: {backtest_results.get('total_return', 0):.2f}%")
        print(f"   📊 Sharpe: {backtest_results.get('sharpe_ratio', 0):.3f}")
        print(f"   ? Trades: {backtest_results.get('total_trades', 0)}")
        print(f"   🎯 Win Rate: {backtest_results.get('win_rate', 0):.1f}%")
        
    else:
        print("⚠️ Aucun signal d'achat trouvé - impossible de faire le backtest")
        print("💡 Vérifiez que:")
        print("   1. Les indicateurs sont correctement calculés")
        print("   2. Les conditions de signal ne sont pas trop restrictives")
        print("   3. La période de données contient des opportunités")
        
else:
    print("❌ Pas de données de signaux disponibles pour le backtesting")
    print("💡 Exécutez d'abord la cellule de génération des signaux")

## 7. Analyse Avancée des Performances

In [ ]:
# Analyse avancée des performances avec VectorBT
if 'backtest_results' in locals():
    print("📊 ANALYSE AVANCÉE DES PERFORMANCES")
    print("=" * 50)
    
    # 1. Utilisation des capacités VectorBT si disponibles
    if vbt_available and 'portfolio_obj' in backtest_results:
        print("? ANALYSE VECTORBT PROFESSIONNELLE ACTIVÉE")
        print("=" * 45)
        
        portfolio = backtest_results['portfolio_obj']
        
        # Métriques avancées VectorBT
        try:
            print("\n? MÉTRIQUES AVANCÉES VECTORBT:")
            print("-" * 35)
            
            # Statistiques complètes
            stats = portfolio.stats()
            for key, value in stats.items():
                if isinstance(value, (int, float)):
                    print(f"   {key}: {value:.4f}")
                else:
                    print(f"   {key}: {value}")
            
            # Métriques de risque avancées
            print(f"\n⚠️ ANALYSE DE RISQUE VECTORBT:")
            print(f"   ? VaR (5%): {portfolio.value_at_risk(cutoff=0.05) * 100:.3f}%")
            print(f"   📊 CVaR (5%): {portfolio.conditional_value_at_risk(cutoff=0.05) * 100:.3f}%")
            print(f"   📊 Max Drawdown Duration: {portfolio.drawdowns.max_duration}")
            
            # Analyse des trades si disponible
            trades = portfolio.trades
            if trades.count() > 0:
                print(f"\n🔍 ANALYSE DES TRADES VECTORBT:")
                print(f"   📊 Trades Stats:")
                trade_stats = trades.stats()
                for key, value in trade_stats.items():
                    if isinstance(value, (int, float)):
                        print(f"      {key}: {value:.4f}")
                        
            print("\n🎯 Génération des graphiques VectorBT...")
            # Graphiques VectorBT intégrés
            backtester.plot_vectorbt_analysis(backtest_results)
            
        except Exception as e:
            print(f"⚠️ Erreur analyse VectorBT avancée: {e}")
            print("🔄 Utilisation des métriques de base...")
    
    # 2. Analyse de base (toujours disponible)
    print(f"\n📊 ANALYSE DE BASE CONSOLIDÉE")
    print("-" * 35)
    
    strategy_return = backtest_results['total_return']
    buy_hold_return = backtest_results['buy_hold_return']
    outperformance = strategy_return - buy_hold_return
    max_dd = backtest_results['max_drawdown']
    
    print(f"💰 Performance Strategy: {strategy_return:.2f}%")
    print(f"📊 Performance Buy & Hold: {buy_hold_return:.2f}%")
    print(f"🚀 Alpha (Surperformance): {outperformance:+.2f}%")
    print(f"📉 Drawdown Maximum: {max_dd:.2f}%")
    print(f"⚖️ Sharpe Ratio: {backtest_results['sharpe_ratio']:.3f}")
    print(f"📊 Calmar Ratio: {backtest_results['calmar_ratio']:.3f}")
    
    # 3. Évaluation de la stratégie
    print(f"\n🎯 ÉVALUATION DE LA STRATÉGIE")
    print("-" * 30)
    
    if outperformance > 0:
        print("✅ La stratégie SURPERFORME le Buy & Hold!")
        if backtest_results['sharpe_ratio'] > 1.0:
            print("🌟 Excellent ratio rendement/risque (Sharpe > 1.0)")
        elif backtest_results['sharpe_ratio'] > 0.5:
            print("? Bon ratio rendement/risque")
        else:
            print("⚠️ Ratio rendement/risque moyen")
    else:
        print("⚠️ La stratégie sous-performe le Buy & Hold")
        print("? Considérez l'optimisation des paramètres")
    
    if max_dd < 10:
        print("✅ Risque contrôlé (Drawdown < 10%)")
    elif max_dd < 20:
        print("⚠️ Risque modéré (Drawdown < 20%)")
    else:
        print("🔴 Risque élevé (Drawdown > 20%)")
    
    # 4. Recommandations
    print(f"\n💡 RECOMMANDATIONS")
    print("-" * 18)
    
    total_trades = backtest_results.get('total_trades', 0)
    win_rate = backtest_results.get('win_rate', 0)
    
    if total_trades < 10:
        print("⚠️ Peu de trades - Considérez ajuster les conditions d'entrée")
    elif total_trades > 100:
        print("📊 Beaucoup de trades - Vérifiez les coûts de transaction")
    else:
        print("✅ Nombre de trades raisonnable")
    
    if win_rate > 60:
        print("✅ Excellent taux de réussite")
    elif win_rate > 40:
        print("👍 Bon taux de réussite")
    else:
        print("⚠️ Taux de réussite faible - Optimisez les signaux")
    
    print(f"\n🏁 Analyse terminée - {'VectorBT' if vbt_available and 'portfolio_obj' in backtest_results else 'Mode de base'} utilisé")
    
else:
    print("❌ Aucun résultat de backtest disponible")
    print("💡 Exécutez d'abord la cellule de backtesting")

## 8. Table des Signaux pour Sauvegarde

Selon les spécifications du README, nous générons une table avec les signaux de trading sans gestion de risque complexe.

In [ ]:
# Génération de la table finale des signaux pour sauvegarde en Gold
if 'df_signals' in locals() and df_signals.height > 0:
    print("💾 GÉNÉRATION DE LA TABLE DES SIGNAUX POUR SAUVEGARDE")
    print("=" * 55)
    
    # Création de la table finale avec les informations essentielles
    signals_table = df_signals.select([
        "datetime",
        "open", "high", "low", "close", "volume",
        "buy_signal", "sell_signal",
        # Indicateurs clés utilisés dans la stratégie  
        "ema_12", "ema_26",
        "rsi_14",
        "supertrend_10_3.0", "supertrend_dir_10_3.0",
        "bb_upper_20_2", "bb_middle_20_2", "bb_lower_20_2",
        "macd_12_26_9", "macd_signal_12_26_9",
        # Conditions intermédiaires pour debugging
        "ema_bullish_cross", "ema_bearish_cross",
        "supertrend_bullish", "supertrend_bearish_flip",
        "rsi_neutral_zone", "rsi_oversold",
        "bb_near_middle", "macd_bullish"
    ]).with_columns([
        # Métadonnées pour la table Gold
        pl.lit("smart_momentum").alias("strategy_name"),
        pl.lit("v1.0").alias("strategy_version"),
        pl.lit(config.SYMBOL).alias("symbol"),
        pl.lit(config.TIMEFRAME).alias("timeframe"),
        pl.lit(datetime.utcnow().isoformat()).alias("generated_at"),
        
        # Signal consolidé (simplifié pour usage)
        pl.when(pl.col("buy_signal") == True).then("BUY")
         .when(pl.col("sell_signal") == True).then("SELL")
         .otherwise("HOLD").alias("signal"),
         
        # Calcul du nombre de conditions remplies pour qualité du signal
        (
            pl.col("ema_bullish_cross").cast(pl.Int8) +
            pl.col("supertrend_bullish").cast(pl.Int8) +  
            pl.col("rsi_neutral_zone").cast(pl.Int8) +
            pl.col("bb_near_middle").cast(pl.Int8) +
            pl.col("macd_bullish").cast(pl.Int8)
        ).alias("signal_strength_buy"),
        
        (
            pl.col("supertrend_bearish_flip").cast(pl.Int8) +
            (pl.col("ema_bearish_cross") & pl.col("rsi_oversold")).cast(pl.Int8)
        ).alias("signal_strength_sell")
    ])
    
    # Statistiques de la table
    total_rows = signals_table.height
    buy_signals = signals_table.filter(pl.col("signal") == "BUY").height
    sell_signals = signals_table.filter(pl.col("signal") == "SELL").height
    hold_periods = signals_table.filter(pl.col("signal") == "HOLD").height
    
    print(f"📊 TABLE DES SIGNAUX GÉNÉRÉE:")
    print(f"   📋 Total de lignes: {total_rows:,}")
    print(f"   📈 Signaux BUY: {buy_signals} ({buy_signals/total_rows*100:.2f}%)")
    print(f"   📉 Signaux SELL: {sell_signals} ({sell_signals/total_rows*100:.2f}%)")
    print(f"   ⚪ Périodes HOLD: {hold_periods} ({hold_periods/total_rows*100:.2f}%)")
    
    # Aperçu de la table
    print(f"\n🔍 APERÇU DE LA TABLE (dernières 10 lignes avec signaux):")
    sample_signals = signals_table.filter(pl.col("signal") != "HOLD").tail(10)
    
    if sample_signals.height > 0:
        display_cols = ["datetime", "close", "signal", "signal_strength_buy", "signal_strength_sell", 
                       "ema_12", "ema_26", "rsi_14"]
        print(sample_signals.select(display_cols))
    else:
        print("   Aucun signal actif dans les dernières données")
    
    # Sauvegarde optionnelle (décommentez pour sauvegarder)
    # strategy.save_strategy_results(signals_table)
    
    print(f"\n✅ Table des signaux prête pour sauvegarde en zone Gold")
    print(f"📊 Colonnes: {signals_table.width}")
    print(f"📅 Période: {signals_table['datetime'].min()} → {signals_table['datetime'].max()}")
    
else:
    print("❌ Aucune donnée de signaux disponible pour créer la table")
    print("💡 Exécutez d'abord les cellules de génération des signaux")

---

## ✅ Stratégie Smart Momentum - Architecture Lakehouse Gold

### 🎯 **Résumé de la Stratégie Réorganisée**

La stratégie **Smart Momentum** utilise maintenant l'architecture data lakehouse complète :

1. **📊 Source** : Feature Store Gold avec tous les indicateurs pré-calculés
2. **🎯 Logique** : Combinaison de 5 indicateurs techniques optimisés
3. **🧮 Backtesting** : VectorBT pour analyse de performance avancée
4. **💾 Output** : Table de signaux sauvegardée en zone Gold

### 🏗️ **Conformité avec l'Architecture README**

- **🥉 Bronze** : Données brutes des API (Binance)
- **🥈 Silver** : Données nettoyées et structurées
- **🥇 Gold** : Feature Store + Tables de signaux de stratégies
- **📊 Marts** : Tableaux de bord et analyses spécialisées

### 🚀 **Avantages de cette Version**

- **⚡ Performance** : Pas de recalcul d'indicateurs, utilisation directe du Feature Store
- **🎯 Focus** : Génération de signaux pure sans gestion de risque complexe
- **📊 Backtesting Professionnel** : VectorBT pour métriques avancées (Sharpe, Calmar, VaR, etc.)
- **🔄 Extensibilité** : Base solide pour ajouter d'autres stratégies
- **📈 Traçabilité** : Architecture lakehouse garantit la reproductibilité

### 📈 **Workflow Simplifié**

1. **📥 Chargement** : Features depuis Gold Feature Store
2. **🎯 Signaux** : Génération basée sur 5 indicateurs techniques
3. **🧮 Backtesting** : Analyse avec VectorBT (performance, risque, trades)
4. **💾 Sauvegarde** : Table de signaux en zone Gold

### 🎉 **Votre pipeline de trading est maintenant intégré à l'architecture lakehouse !**

---

## 🚀 Test VectorBT - Environnement Python 3.10

Cette section teste la nouvelle configuration avec VectorBT fonctionnel.

In [ ]:
# Test de l'environnement VectorBT optimisé
print("🧪 TEST DE L'ENVIRONNEMENT VECTORBT OPTIMISÉ")
print("=" * 50)

# 1. Test d'import et versions
print("📦 Vérification des versions:")
try:
    import vectorbt as vbt
    import numpy as np
    import pandas as pd
    import sys
    
    print(f"✅ Python: {sys.version.split()[0]}")
    print(f"✅ VectorBT: {vbt.__version__}")
    print(f"✅ NumPy: {np.__version__}")
    print(f"✅ Pandas: {pd.__version__}")
    
    # 2. Test de performance VectorBT avec données simulées
    print(f"\n🎯 Test de performance VectorBT:")
    
    # Génération de données de test
    dates = pd.date_range('2023-01-01', '2023-12-31', freq='D')
    np.random.seed(42)
    prices = pd.Series(
        100 + np.cumsum(np.random.randn(len(dates)) * 0.02),
        index=dates
    )
    
    # Signaux de test simples
    buy_signals = prices.rolling(20).mean() > prices.rolling(50).mean()
    sell_signals = prices.rolling(20).mean() < prices.rolling(50).mean()
    
    # Test backtesting VectorBT
    portfolio = vbt.Portfolio.from_signals(
        prices,
        buy_signals,
        sell_signals,
        init_cash=10000,
        fees=0.001
    )
    
    print(f"   💰 Rendement total: {portfolio.total_return() * 100:.2f}%")
    print(f"   📊 Sharpe ratio: {portfolio.sharpe_ratio():.3f}")
    print(f"   🔄 Nombre de trades: {portfolio.trades.count()}")
    print(f"   ⚖️ Calmar ratio: {portfolio.calmar_ratio():.3f}")
    
    print("\n🎉 VECTORBT FONCTIONNE PARFAITEMENT !")
    print("✅ Environnement Python 3.10 optimisé")
    print("✅ Toutes les métriques avancées disponibles")
    print("✅ Backtesting professionnel activé")
    
except ImportError as e:
    print(f"❌ Erreur d'import: {e}")
except Exception as e:
    print(f"⚠️ Erreur lors du test: {e}")

# 3. Configuration recommandée
print(f"\n⚙️ CONFIGURATION RECOMMANDÉE:")
print("   🐍 Python 3.10.x (Compatible VectorBT)")
print("   📦 VectorBT 0.25.x (Dernière stable)")
print("   🔢 NumPy < 1.25 (Compatibilité)")
print("   🐼 Pandas 2.x (Performance)")

print(f"\n🚀 Environnement prêt pour le backtesting professionnel !")